In [26]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import seaborn as sns
from sklearn.preprocessing import scale
from sklearn.metrics import accuracy_score

In [27]:
X=df.drop(columns='GRADE')
y=df['GRADE']
print('X:',X.shape)
display(X.head())
print('y:',y.shape)
print(y.head())

X: (32, 3)


,GPA,TUCE,PSI
0,2.66,20.0,0.0
1,2.89,22.0,0.0
2,3.28,24.0,0.0
3,2.92,12.0,0.0
4,4.00,21.0,0.0


y: (32,)
0    0.0
1    0.0
2    0.0
3    0.0
4    1.0
Name: GRADE, dtype: float64


In [28]:
X_dumm = pd.get_dummies(X,drop_first=True)
print('X_dumm:',X_dumm.shape)
display(X_dumm.head())

X_dumm: (32, 3)


,GPA,TUCE,PSI
0,2.66,20.0,0.0
1,2.89,22.0,0.0
2,3.28,24.0,0.0
3,2.92,12.0,0.0
4,4.00,21.0,0.0


In [29]:
X_dumm_c = sm.add_constant(X_dumm)
model = sm.Logit(y,X_dumm_c)
results = model.fit()
print(results.summary())

Optimization terminated successfully.
         Current function value: 0.402801
         Iterations 7
                           Logit Regression Results                           
Dep. Variable:                  GRADE   No. Observations:                   32
Model:                          Logit   Df Residuals:                       28
Method:                           MLE   Df Model:                            3
Date:                Thu, 08 Jun 2023   Pseudo R-squ.:                  0.3740
Time:                        15:54:29   Log-Likelihood:                -12.890
converged:                       True   LL-Null:                       -20.592
Covariance Type:            nonrobust   LLR p-value:                  0.001502
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const        -13.0213      4.931     -2.641      0.008     -22.687      -3.356
GPA            2.8261      1.

In [30]:
y_pred = results.predict(X_dumm_c)
print('Predicted y:')
print(y_pred.head())
y_pred_bin = (y_pred >= 0.5).astype('int')
print(y_pred_bin.head())

Predicted y:
0    0.026578
1    0.059501
2    0.187260
3    0.025902
4    0.569893
dtype: float64
0    0
1    0
2    0
3    0
4    1
dtype: int32


In [31]:
acc = accuracy_score(y,y_pred_bin)
print('Hitting ratio:',acc)

Hitting ratio: 0.8125


In [32]:
print('Odds ratio')
print(np.exp(results.params))

Odds ratio
const     0.000002
GPA      16.879715
TUCE      1.099832
PSI      10.790732
dtype: float64


In [33]:
X_scaled_ar = scale(X_dumm)
X_scaled = pd.DataFrame(X_scaled_ar,columns=X_dumm.columns)
X_scaled_c = sm.add_constant(X_scaled)
model = sm.Logit(y,X_scaled_c)
results_scaled = model.fit()
print(results_scaled.summary())

Optimization terminated successfully.
         Current function value: 0.402801
         Iterations 7
                           Logit Regression Results                           
Dep. Variable:                  GRADE   No. Observations:                   32
Model:                          Logit   Df Residuals:                       28
Method:                           MLE   Df Model:                            3
Date:                Thu, 08 Jun 2023   Pseudo R-squ.:                  0.3740
Time:                        15:54:30   Log-Likelihood:                -12.890
converged:                       True   LL-Null:                       -20.592
Covariance Type:            nonrobust   LLR p-value:                  0.001502
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -1.0836      0.557     -1.944      0.052      -2.176       0.009
GPA            1.2982      0.

In [34]:
print(results_scaled.params.sort_values(key=np.abs,ascending=False))

GPA      1.298210
PSI      1.180015
const   -1.083627
TUCE     0.365412
dtype: float64


In [35]:
X_test = pd.DataFrame([[3.3,25,1],[2.1,15,0],],columns=X_dumm.columns)

print('X for prediction:')
display(X_test)

X for prediction:


,GPA,TUCE,PSI
0,3.3,25,1
1,2.1,15,0


In [36]:
X_test_c = sm.add_constant(X_test,has_constant='add')
y_test = results.predict(X_test_c)
print('Predicted y:')
print(y_test)

Predicted y:
0    0.743159
1    0.003473
dtype: float64
